<div align="center">
    <h1>DS-210: Programming for Data Science</h1>
    <h1>Lecture 20</h1>
</div>

# NDArray:  Rust's answer to numpy

## Key similarities

* Arrays have single element types
* Arrays can have arbitrary dimensions
* Arrays can have arbitrary strides
* Indexing starts at 0
* Default ordering is row-major (more on that below)
* Arithmetic operators (+, -, \*, /) perform element-wise operations
* Arrays that are not views are contiguous in memory
* Many cheap operations that return views into the array instead of copying data


## Some important differences

* Numpy arrays and views are all mutable and can all change the contents of an array.  NDarrays have flavors that can change contents, flavors that cannot, and flavors that make copies when things change.
* Numpy arrays are always dynamic in their number of dimensions.  NDarrays can be static or dynamic.
* Slices with negative steps behave differently (more on that later)

## To use it

[dependencies]  
ndarray="0.15.6"  

in your Cargo.toml file


## Why use NDarray over Vec or array?

It is easier to do a bunch of things like:

* Data Cleaning and Preprocessing offering functions like fill
* Statistics built in (lots of math functions come with it)
* Machine learning: Used in many of the ML libraries written in Rust
* Linear Algebra: Built in methods like matrix inversion, multiplication and decomposition.

In [10]:
:dep ndarray = { version = "^0.15.6" }

use ndarray::prelude::*;

fn main() {
    let a = array![
                [1.,2.,3.], 
                [4.,5.,6.],
            ]; 
    assert_eq!(a.ndim(), 2);         // get the number of dimensions of array a
    assert_eq!(a.len(), 6);          // get the number of elements in array a
    assert_eq!(a.shape(), [2, 3]);   // get the shape of array a
    assert_eq!(a.is_empty(), false); // check if the array has zero elements

    println!("{:?}", a);
}

main();

[[1.0, 2.0, 3.0],
 [4.0, 5.0, 6.0]], shape=[2, 3], strides=[3, 1], layout=Cc (0x5), const ndim=2


### Great side by side comparison at https://docs.rs/ndarray/latest/ndarray/doc/ndarray_for_numpy_users/index.html

### Array Creation

|NumPy | ndarray | Notes |
|:-:|:-:|:-:|
| np.array([[1.,2.,3.], [4.,5.,6.]])| array![[1.,2.,3.], [4.,5.,6.]] or arr2(&[[1.,2.,3.], [4.,5.,6.]]) | 2×3 floating-point array literal|
|np.arange(0., 10., 0.5) | Array::range(0., 10., 0.5) | create a 1-D array with values 0., 0.5, …, 9.5|
|np.linspace(0., 10., 11) |	Array::linspace(0., 10., 11) | create a 1-D array with 11 elements with values 0., …, 10.|
|np.logspace(2.0, 3.0, num=4, base=10.0) | Array::logspace(10.0, 2.0, 3.0, 4) | create a 1-D array with 4 elements with values 100., 215.4, 464.1, 1000.|
|np.geomspace(1., 1000., num=4) | Array::geomspace(1e0, 1e3, 4) | create a 1-D array with 4 elements with values 1., 10., 100., 1000.|
|np.ones((3, 4, 5)) | Array::ones((3, 4, 5)) | create a 3×4×5 array filled with ones (inferring the element type)|
|np.zeros((3, 4, 5)) | Array::zeros((3, 4, 5)) | create a 3×4×5 array filled with zeros (inferring the element type)|
|np.zeros((3, 4, 5), order='F')	| Array::zeros((3, 4, 5).f()) | create a 3×4×5 array with Fortran (column-major) memory layout filled with zeros (inferring the element type)|
|np.zeros_like(a, order='C') | Array::zeros(a.raw_dim()) | create an array of zeros of the shape shape as a, with row-major memory layout (unlike NumPy, this infers the element type from context instead of duplicating a’s element type)|
|np.full((3, 4), 7.) | Array::from_elem((3, 4), 7.) | create a 3×4 array filled with the value 7.|
|np.array([1, 2, 3, 4]).reshape((2, 2)) | Array::from_shape_vec((2, 2), vec![1, 2, 3, 4])? or .into_shape((2,2).unwrap()| create a 2×2 array from the elements in the Vec|

In [11]:
let a:Array<f64, Ix1> = Array::linspace(0., 4.5, 10);
println!("{:?}", a);

let b:Array<f64, _> = a.into_shape((2,5)).unwrap();
println!("{:?}", b);

let c:Array<f64, _> = Array::zeros((3,4).f());
println!("{:?}", c);

[0.0, 0.5, 1.0, 1.5, 2.0, 2.5, 3.0, 3.5, 4.0, 4.5], shape=[10], strides=[1], layout=CFcf (0xf), const ndim=1
[[0.0, 0.5, 1.0, 1.5, 2.0],
 [2.5, 3.0, 3.5, 4.0, 4.5]], shape=[2, 5], strides=[5, 1], layout=Cc (0x5), const ndim=2
[[0.0, 0.0, 0.0, 0.0],
 [0.0, 0.0, 0.0, 0.0],
 [0.0, 0.0, 0.0, 0.0]], shape=[3, 4], strides=[1, 3], layout=Ff (0xa), const ndim=2


## What does the printout mean?

* The values of the array
* The shape of the array, most important dimension first
* The stride (always 1 for arrays in the last dimension, but can be different for views)
* The layout (storage order, view order)
* The number of dimensions

## Indexing and Slicing

|NumPy | ndarray | Notes|
|:-:|:-:|:-:|
|a[-1] | a[a.len() - 1] | access the last element in 1-D array a|
|a[1, 4] | a[[1, 4]] | access the element in row 1, column 4|
|a[1] or a[1, :, :] | a.slice(s![1, .., ..]) or a.index_axis(Axis(0), 1) | get a 2-D subview of a 3-D array at index 1 of axis 0|
|a[0:5] or a[:5] or a[0:5, :] | a.slice(s![0..5, ..]) or a.slice(s![..5, ..]) or a.slice_axis(Axis(0), Slice::from(0..5)) | get the first 5 rows of a 2-D array|
|a[-5:] or a[-5:, :] | a.slice(s![-5.., ..]) or a.slice_axis(Axis(0), Slice::from(-5..)) | get the last 5 rows of a 2-D array|
|a[:3, 4:9]	| a.slice(s![..3, 4..9]) | columns 4, 5, 6, 7, and 8 of the first 3 rows|
|a[1:4:2, ::-1]	| a.slice(s![1..4;2, ..;-1]) | rows 1 and 3 with the columns in reverse order|

In [12]:
use ndarray::{s};
{
    // Create a 3-dimensional array (2x3x4)
    let mut array = Array::from_shape_fn((2, 3, 4), |(i, j, k)| {
        (i * 100 + j * 10 + k) as f32
    });

    // Print the original 3-dimensional array
    println!("Original 3D array:\n{:?}", array);

    // Create a 2-dimensional slice (taking the first 2D layer)
    let mut slice = array.slice_mut(s![1.., 0, ..]);

    // Print the 2-dimensional slice
    println!("2D slice:\n{:?}", slice);

    // Create a 1-dimensional slice (taking the first 2D and 3D layer)
    let slice2 = array.slice(s![1, 0, ..]);

    // Print the 1-dimensional slice
    println!("1D slice:\n{:?}", slice2);

}

Original 3D array:
[[[0.0, 1.0, 2.0, 3.0],
  [10.0, 11.0, 12.0, 13.0],
  [20.0, 21.0, 22.0, 23.0]],

 [[100.0, 101.0, 102.0, 103.0],
  [110.0, 111.0, 112.0, 113.0],
  [120.0, 121.0, 122.0, 123.0]]], shape=[2, 3, 4], strides=[12, 4, 1], layout=Cc (0x5), const ndim=3
2D slice:
[[100.0, 101.0, 102.0, 103.0]], shape=[1, 4], strides=[0, 1], layout=CFcf (0xf), const ndim=2
1D slice:
[100.0, 101.0, 102.0, 103.0], shape=[4], strides=[1], layout=CFcf (0xf), const ndim=1


()

## Examining the array parameters

|NumPy | ndarray | Notes|
|:-:|:-:|:-:|
|np.ndim(a) or a.ndim | a.ndim() | get the number of dimensions of array a|
|np.size(a) or a.size | a.len()	| get the number of elements in array a|
|np.shape(a) or a.shape	| a.shape() or a.dim() | get the shape of array a|
|a.shape[axis] | a.len_of(Axis(axis)) | get the length of an axis|
|a.strides | a.strides() | get the strides of array a|
|np.size(a) == 0 or a.size == 0	| a.is_empty() | check if the array has zero elements|


## Simple Math

|NumPy | ndarray | Notes|
|:-:|:-:|:-:|
|a.transpose() or a.T | a.t() or a.reversed_axes() | transpose of array a (view for .t() or by-move for .reversed_axes())|
|mat1.dot(mat2) | mat1.dot(&mat2) |2-D matrix multiply|
|mat.dot(vec) | mat.dot(&vec) | 2-D matrix dot 1-D column vector|
|vec.dot(mat) | vec.dot(&mat) | 1-D row vector dot 2-D matrix |
|vec1.dot(vec2) | vec1.dot(&vec2) | vector dot product|
|a * b, a + b, etc. | a * b, a + b, etc. | element-wise arithmetic operations|
|a\**3 | a.mapv(\|a\| a.powi(3)) | element-wise power of 3|
|np.sqrt(a) | a.mapv(f64::sqrt) | element-wise square root for f64 array |
|(a>0.5) | a.mapv(\|a\| a > 0.5)| array of bools of same shape as a with true where a > 0.5 and false elsewhere|
|np.sum(a) or a.sum() | a.sum() | sum the elements in a|
| np.sum(a, axis=2) or a.sum(axis=2) | a.sum_axis(Axis(2)) | sum the elements in a along axis 2|
|np.mean(a) or a.mean() | a.mean().unwrap() | calculate the mean of the elements in f64 array a|
|np.mean(a, axis=2) or a.mean(axis=2) | a.mean_axis(Axis(2)) | calculate the mean of the elements in a along axis 2|
|np.allclose(a, b, atol=1e-8) | a.abs_diff_eq(&b, 1e-8) |  check if the arrays’ elementwise differences are within an absolute tolerance (it requires the approx feature-flag)|
| np.diag(a) | a.diag() | view the diagonal of a|


### mapv vs map:  mapv iterates over the values of the array, map iterates over mutable references of the array

### Array0 to Array6 also defined in addition to Array as special cases with fixed dimensions instead dynamically defined dimensions.

In [13]:
{
    // Create a 2-dimensional array (3x4)
    let mut a = Array::from_shape_fn((3, 4), |(j, k)| {
        (j * 10 + k) as f32
    });

    // Print the original 2-dimensional array
    println!("Original 2D array:\n{:?}", a);

    let b = a * 2.0;
    println!("b:\n{:?}", b);

    let c = b.mapv(|v| v>24.8);
    println!("c:\n{:?}", c);
}

Original 2D array:
[[0.0, 1.0, 2.0, 3.0],
 [10.0, 11.0, 12.0, 13.0],
 [20.0, 21.0, 22.0, 23.0]], shape=[3, 4], strides=[4, 1], layout=Cc (0x5), const ndim=2
b:
[[0.0, 2.0, 4.0, 6.0],
 [20.0, 22.0, 24.0, 26.0],
 [40.0, 42.0, 44.0, 46.0]], shape=[3, 4], strides=[4, 1], layout=Cc (0x5), const ndim=2
c:
[[false, false, false, false],
 [false, false, false, true],
 [true, true, true, true]], shape=[3, 4], strides=[4, 1], layout=Cc (0x5), const ndim=2


()

## Type Conversions

* std::convert::From ensures lossless, safe conversions at compile-time and is generally recommended.
* std::convert::TryFrom can be used for potentially unsafe conversions. It will return a Result which can be handled or unwrap()ed to panic if any value at runtime cannot be converted losslessly.


|NumPy | ndarray | Notes|
|:-:|:-:|:-:|
|a.astype(np.float32) | a.mapv(\|x\| f32::from(x)) | convert array to f32.  Only use if can't fail |
|a.astype(np.int32) | a.mapv(\|x\| i32::from(x)) | convert array to i32.  Only use if can't fail |
|a.astype(np.uint8) | a.mapv(\|x\| u8::try_from(x).unwrap()) | try to convert to u8 array, panic if any value cannot be converted lossless at runtime (e.g. negative value)|
|a.astype(np.int32) | a.mapv(\|x\| x as i32) | convert to i32 array with “saturating” conversion; care needed because it can be a lossy conversion or result in non-finite values!|



## Linear Algebra

|NumPy | ndarray | Notes|
|:-:|:-:|:-:|
|numpy.linalg.inv(a) | a.inv() | Invert matrix a.  Must be square|
|numpy.linalg.eig(a) | a.eig() | Compute eigenvalues and eigenvectors of matrix.  Must be square|
|numpy.linalg.svd(a) | a.svd(true, true) | Compute the Singular Value Decomposition of matrix|
|numpy.linalg.det(a) | a.det() | Compute the determinant of a matrix. Must be square|

In [14]:
:dep ndarray = { version = "^0.15" }
// This is the MAC version
:dep ndarray-linalg = { version = "^0.15", features = ["netlib"] }
// This is the linux verison
//:dep ndarray-linalg = { version = "^0.15", features = ["openblas"] }

use ndarray::array;
use ndarray_linalg::*;

{
    // Create a 2D square matrix (3x3)
    let matrix = array![
        [1.0, 2.0, 3.0],
        [0.0, 1.0, 4.0],
        [5.0, 6.0, 0.0]
    ];

    // Compute the eigenvalues and eigenvectors
    match matrix.eig() {
        Ok((eigenvalues, eigenvectors)) => {
            println!("Eigenvalues:\n{:?}", eigenvalues);
            println!("Eigenvectors:\n{:?}", eigenvectors);
        },
        Err(e) => println!("Failed to compute eigenvalues and eigenvectors: {:?}", e),
    }
}


Eigenvalues:
[Complex { re: 7.256022422687388, im: 0.0 }, Complex { re: -0.026352822204034426, im: 0.0 }, Complex { re: -5.229669600483354, im: 0.0 }], shape=[3], strides=[1], layout=CFcf (0xf), const ndim=1
Eigenvectors:
[[Complex { re: -0.4992701697014973, im: 0.0 }, Complex { re: -0.7576983872742932, im: 0.0 }, Complex { re: -0.2257801627715908, im: 0.0 }],
 [Complex { re: -0.4667420094775666, im: 0.0 }, Complex { re: 0.6321277120502755, im: 0.0 }, Complex { re: -0.5263484547676879, im: 0.0 }],
 [Complex { re: -0.7299871192254567, im: 0.0 }, Complex { re: -0.162196515314045, im: 0.0 }, Complex { re: 0.8197442419819129, im: 0.0 }]], shape=[3, 3], strides=[1, 3], layout=Ff (0xa), const ndim=2


()

## NDArray and Neural networks

Much of neural network training and evaluation comes down to vector operations in which ndarray excels.
Let's take a look at some common operations needed by neural network training


## Forward propagation

* Weights would be represented by an Array2<f32> data structure with the dimensions being the number of input neurons and the number of middle neurons
* For example if you have 100 input neurons and 50 2nd layer neurons then you need 100X50 matrix of weights
* The operation to calculate the value in the 50 middle neurons would be a matrix vector multiplication between the input (a vector) and the weight matrix
* The output would be passed through an S-shaped function to limit results between 0 and 1.  Common choices for this function include the sigmoid or logistical functions
* Sigmoid function: $1 / (1 + e^{-x})$
    

In [15]:
:dep ndarray = { version = "^0.15" }
:dep rand = { version = "0.8" }

use ndarray::Array;
use ndarray::Array2;
use rand::Rng;

fn populate_array(arr: &mut Array2<f32>, m:usize, n:usize) {
    let mut rng = rand::thread_rng();
    for i in 0..m {
        for j in 0..n {
            arr[(i, j)] = rng.gen_range(0.0..0.2);
        }
    }
}

fn sigmoid(arr: &mut Array2<f32>) -> Array2<f32>{
    arr.mapv(|x| 1.0 / (1.0 + (-x).exp()))
}

fn sigmoid_derivative(x: &Array2<f32>) -> Array2<f32> {
    x * &(1.0 - x)
}

// Just generate some randomly populated 1XM and MXN arrays and then compute the dot product for them
let input_size = 100;
let hidden_size = 50;
let mut input:Array2<f32> = Array2::zeros((1,input_size));
populate_array(&mut input, 1, input_size);
let mut weights:Array2<f32> = Array2::zeros((input_size, hidden_size));
populate_array(&mut weights, input_size, hidden_size);


let mut hidden:Array2<f32> = input.dot(&weights);
println!("Inputs: {:?}\n\n Weights:{:?}\n\n Hidden:{:?}\n\n", input, weights, hidden);

// And now apply a sigmoid function to it 
let mut activation = sigmoid(&mut hidden);
println!("Sigmoid Activation:{:?}", activation);


Inputs: [[0.009873248, 0.08901768, 0.14424205, 0.09695544, 0.10035365, 0.09715257, 0.08848093, 0.14204226, 0.0040730718, 0.19753383, 0.028625393, 0.01883824, 0.17018607, 0.12178662, 0.16322966, 0.030388761, 0.17308621, 0.1302458, 0.14377129, 0.121864654, 0.18884318, 0.08695419, 0.02837403, 0.0511333, 0.14942634, 0.1765658, 0.007947779, 0.017442537, 0.13617228, 0.023305392, 0.05701082, 0.12093764, 0.10127018, 0.18754755, 0.059944727, 0.034534194, 0.12706648, 0.19897433, 0.11871784, 0.09938734, 0.18468015, 0.008186365, 0.082978584, 0.009964633, 0.19318698, 0.06574192, 0.075774334, 0.06165128, 0.03515465, 0.11850148, 0.045211244, 0.12850164, 0.03401959, 0.13831656, 0.1878591, 0.07012194, 0.06859088, 0.00933063, 0.11924412, 0.17604207, 0.1529017, 0.02655585, 0.08914802, 0.14398776, 0.1374756, 0.12707143, 0.09420004, 0.14293563, 0.12375946, 0.1889296, 0.074413635, 0.04042814, 0.1257735, 0.08427997, 0.044573497, 0.056812238, 0.109331205, 0.09426756, 0.09883197, 0.1621439, 0.12196632, 0.00385

## Backwards propagation

* In backward propagation we want to use the error between our forward pass and the expected value to update the weights.
* Start by subtracting the expected output vector from the calculated one.  This is the derivative of the last step and it's a simple subtraction.  It gives the error for every output and by how much we want to move the predicted output.
* We need to figure out how to change the previous step, so since the predicted output was calculated by the activation function, we need to see how its input should be affected to reduce the error.  This means the error from the previous step needs to be passed to the derivative of the activation function.
* Now we have the desired deltas for the inputs to the activation function. We also have the system inputs that produced the activation function inputs given our current weights.  So we need to adjust the weights in a direction pointed by the deltas.
* We typically make adjustment in small increments to avoid our function jumping around too much as a single sample can be misleading.

In [16]:
// Subtracting 2 vectors to calculate the error

let mut target:Array2<f32> = Array2::zeros((1, hidden_size));
populate_array(&mut target, 1, hidden_size);

let mut error = target - &activation;
let mut delta = sigmoid_derivative(&mut error);
println!("Error:{:?}\n\n Delta{:?}\n\n", error, delta);
// And finally we 'd have to update the weights.  First calculate the deltas by using the dot product 
// of the error and original output
let weight_updates = input.t().dot(&delta);
println!("Weight updates:{:?}\n\n", weight_updates);

weights = weights + &(weight_updates * 0.05);
println!("New weights:{:?}\n\n", weights);


Error:[[-0.7295941, -0.7028239, -0.7431128, -0.5878457, -0.7221791, -0.64208263, -0.5430765, -0.6632087, -0.5696637, -0.5686352, -0.61894614, -0.58176875, -0.5709656, -0.68711376, -0.5635445, -0.6830436, -0.5562143, -0.60204583, -0.54988164, -0.62797344, -0.73132956, -0.6170935, -0.70532304, -0.651486, -0.6782821, -0.5426787, -0.7152052, -0.60126984, -0.62929827, -0.6250593, -0.60553133, -0.6821726, -0.547711, -0.55683285, -0.5598092, -0.6052464, -0.5637011, -0.71052176, -0.57987, -0.60840964, -0.56382525, -0.5410644, -0.55246305, -0.5755404, -0.5943817, -0.6473886, -0.71354485, -0.5389118, -0.7299671, -0.5833124]], shape=[1, 50], strides=[50, 1], layout=CFcf (0xf), const ndim=2

 Delta[[-1.2619017, -1.1967853, -1.2953295, -0.9334082, -1.2437218, -1.0543528, -0.83800864, -1.1030545, -0.8941805, -0.89198124, -1.0020404, -0.92022365, -0.89696723, -1.159239, -0.88112694, -1.1495922, -0.8655886, -0.96450496, -0.85225147, -1.0223241, -1.2661725, -0.9978979, -1.2028036, -1.0759199, -1.138348

### All of this and more...

### Quick start: https://github.com/rust-ndarray/ndarray/blob/master/README-quick-start.md
### Numpy equivalence: https://docs.rs/ndarray/latest/ndarray/doc/ndarray_for_numpy_users/index.html
### Linear Algebra: https://github.com/rust-ndarray/ndarray-linalg/blob/master/README.md